In [1]:
import numpy as np
import pandas as pd

In [2]:
# 1.a
I1 = np.array([
    [0, 0, 0, 0, 0],
    [0, 1, 1, 1, 0],
    [0, 1, 0, 1, 0],
    [0, 1, 1, 1, 0],
    [0, 0, 0, 0, 0]
])
I2 = np.array([
    [0, 0, 0, 0, 0],
    [0, 0, 1, 0, 0],
    [0, 0, 1, 0, 0],
    [0, 0, 1, 0, 0],
    [0, 0, 0, 0, 0]
])
I3 = np.array([
    [0, 0, 0, 0, 0],
    [1, 1, 1, 0, 0],
    [1, 0, 1, 0, 0],
    [1, 1, 1, 0, 0],
    [0, 0, 0, 0, 0]
])
I4 = np.array([
    [0, 0, 0, 0, 0],
    [0, 0, 0, 1, 0],
    [0, 0, 0, 1, 0],
    [0, 0, 0, 1, 0],
    [0, 0, 0, 0, 0]
])

In [3]:
# Vectorize the images
images = [I1, I2, I3, I4]
vectors = [img.flatten() for img in images]

In [4]:
vectors

[array([0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0,
        0, 0, 0]),
 array([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,
        0, 0, 0]),
 array([0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0,
        0, 0, 0]),
 array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0,
        0, 0, 0])]

In [5]:
n = len(vectors)
d2 = np.zeros((n, n))
for i in range(n):
    for j in range(n):
        diff = vectors[i] - vectors[j]
        d2[i, j] = np.dot(diff, diff)

# Gaussian kernel: k(x, y) = exp(-||x-y||^2 / 2)
K = np.exp(-d2 / 2)

In [6]:
K_df = pd.DataFrame(K, index=[f"I{i+1}" for i in range(n)], columns=[f"I{j+1}" for j in range(n)])

In [7]:
K_df

,I1,I2,I3,I4
I1,1.000000,0.030197,0.018316,0.082085
I2,0.030197,1.000000,0.082085,0.049787
I3,0.018316,0.082085,1.000000,0.004087
I4,0.082085,0.049787,0.004087,1.000000


In [11]:
#1.c
def get_shifts(img):
    right_shift = img[:, :-1].flatten()  # drop far right column
    left_shift  = img[:, 1:].flatten()   # drop far left column
    return right_shift, left_shift

In [ ]:
# list comprehension shenanigans
shifted = [get_shifts(img) for img in images]

In [ ]:
# regular gaussian kernel
def kG(u, v):
    d2 = np.dot(u - v, u - v)
    return np.exp(-d2 / 2)


In [ ]:
n = len(images)
K_shift = np.zeros((n, n))
# for ever image
for i in range(n):
    # get the shifted images
    r_i, l_i = shifted[i]
    # for every shifted imag
    for j in range(n):
        # get the actual shifts
        r_j, l_j = shifted[j]
        # and compute by getting the similarity of the 4 combinations
        # interesting to note that by adding more shifts, combinations would
        # increase exponentially, and so the kernel would be more expensive
        # i think
        K_shift[i, j] = (
            kG(r_i, r_j) +
            kG(l_i, l_j) +
            kG(r_i, l_j) +
            kG(l_i, r_j)
        )


In [ ]:
K_shift_df = pd.DataFrame(
    np.round(K_shift, 6),
    index=[f"I{i+1}" for i in range(n)],
    columns=[f"I{j+1}" for j in range(n)]
)
K_shift_df

,I1,I2,I3,I4
I1,2.036631,0.224565,1.130598,0.198454
I2,0.224565,2.099574,0.498477,1.149361
I3,1.130598,0.498477,2.164170,0.122803
I4,0.198454,1.149361,0.122803,2.099574
